# Loss for Demand Predictions of an Online Store

You are tasked with forecasting the daily demand of an online retailer over a certain time period based on
historical sales data. In addition to the sales data you are also provided with the stock (inventory) that was
available for sale on each day. Therefore, the stock information provides an upper limit for the possible sales
on each day. This small task is not concerned with the development of the model itself but only focuses on
developing and implementing a tailor-suited loss function for this forecasting problem.

- Develop a loss function in PyTorch based on the code snippet below.
- Explain your choice and why it is a good fit for the problem at hand.
- Evaluate the loss function’s performance with the provided dummy model. Adjust the code where
needed to arrive at conclusions. Summarize and explain your findings.

In [1]:
import torch
import torch.nn as nn

In [2]:
# Define the input tensors for stocks and sales
n = 1000
torch.manual_seed(0)
stocks = torch.randint(0, 10, (n,))
demand = torch.poisson(torch.ones(n) * 2.0)
sales = torch.min(demand, stocks)

In [3]:
# display(stocks)
# display(demand)
# display(sales)

```diff
@@ Given the problem statement, one possible approach to defining a loss function would be to penalize both overestimations  and underestimations of demand. In particular, we want to avoid overestimations,  as they might lead to wastage of resources (since stock exceeding demand might go unused), and we also want to avoid underestimations, as they might lead to lost sales opportunities.

@@ One potential loss function to use could be a weighted mean absolute error (WMAE), where we assign different weights to overestimations and underestimations.
```

In [4]:
# Define the loss function
def forecast_loss(predictions, sales, stocks):
    sales = torch.min(sales, stocks)
    overestimation = torch.clamp(predictions - sales, min=0)
    underestimation = torch.clamp(sales - predictions, min=0)
    
    # Set different weights for overestimation and underestimation
    w1, w2 = 0.6, 0.4
    loss = w1 * torch.mean(overestimation) + w2 * torch.mean(underestimation)
    
    return loss

```diff

@@ In the forecast_loss function, we separate the overestimation and underestimation by using torch.clamp. Then we calculate the mean of each part and sum them with different weights. Here, we assign more weight to overestimation (w1=0.6), as we might consider overestimation as more undesirable.

@@ When training the model, the loss function will guide the model to forecast the demand with the least overall error. The model will make a trade-off between underestimation and overestimation according to the weights we have set.

@@ This loss function is useful because it allows us to specify how much we value avoiding overestimation versus avoiding underestimation, which can be tailored to the specific business context. For example, if the cost of overestimation is significantly higher than the cost of underestimation (perhaps because of storage costs, or because unsold items perish), then we would want to set a higher weight for the overestimation term.

```

In [5]:
class MeanModel(nn.Module):
    def __init__(self):
        super(MeanModel, self).__init__()
        self.mean = nn.Parameter(torch.randn(1))
        
    def forward(self, n):
        return self.mean * torch.ones(n) # this a liner model which might not perform better

In [6]:
torch.randn(1) # generates a negative value when multiplied by torch.ones(n), can produce negative outputs

tensor([-0.6617])

In [7]:
# Train the model and optimize using gradient descent
model = MeanModel()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # tested lr rate of 0.001, after 20000 iterations loss stays the same

```diff
@@ The performance of the loss function can be evaluated by running the training loop and checking whether the loss decreases over time, indicating that the model is learning to make better forecasts. Note that in order to make this a fair test, we might need to ensure that the training data includes a variety of situations, including both times when demand exceeds stock, and times when stock exceeds demand.
```

In [8]:
# Print when demand exceeds stock
excess_demand = demand > stocks
print("Times when demand exceeds stock:")
display(torch.nonzero(excess_demand))

# Print when stock exceeds demand
excess_stock = stocks > demand
print("\nTimes when stock exceeds demand:")
display(torch.nonzero(excess_stock))

Times when demand exceeds stock:


tensor([[  3],
        [  4],
        [  9],
        [ 10],
        [ 19],
        [ 29],
        [ 30],
        [ 33],
        [ 36],
        [ 44],
        [ 48],
        [ 50],
        [ 54],
        [ 55],
        [ 65],
        [ 67],
        [ 72],
        [ 73],
        [ 78],
        [ 79],
        [ 85],
        [ 92],
        [105],
        [107],
        [117],
        [124],
        [129],
        [136],
        [138],
        [143],
        [145],
        [150],
        [153],
        [158],
        [160],
        [165],
        [174],
        [176],
        [179],
        [185],
        [194],
        [196],
        [209],
        [222],
        [224],
        [228],
        [229],
        [240],
        [242],
        [246],
        [248],
        [263],
        [270],
        [274],
        [292],
        [294],
        [298],
        [300],
        [306],
        [317],
        [318],
        [324],
        [325],
        [330],
        [331],
        [339],
        [3


Times when stock exceeds demand:


tensor([[  0],
        [  1],
        [  2],
        [  5],
        [  6],
        [  7],
        [  8],
        [ 11],
        [ 12],
        [ 13],
        [ 14],
        [ 15],
        [ 16],
        [ 17],
        [ 18],
        [ 20],
        [ 21],
        [ 23],
        [ 24],
        [ 26],
        [ 27],
        [ 28],
        [ 32],
        [ 34],
        [ 35],
        [ 37],
        [ 39],
        [ 40],
        [ 41],
        [ 42],
        [ 43],
        [ 45],
        [ 46],
        [ 47],
        [ 51],
        [ 53],
        [ 56],
        [ 57],
        [ 58],
        [ 59],
        [ 60],
        [ 61],
        [ 62],
        [ 63],
        [ 64],
        [ 66],
        [ 68],
        [ 70],
        [ 71],
        [ 74],
        [ 75],
        [ 76],
        [ 77],
        [ 81],
        [ 82],
        [ 83],
        [ 84],
        [ 86],
        [ 87],
        [ 88],
        [ 89],
        [ 90],
        [ 93],
        [ 94],
        [ 95],
        [ 96],
        [ 

In [9]:
for i in range(3000): # increase number of iterations from 100 to 3000
    optimizer.zero_grad()
    outputs = model(n)
    loss = forecast_loss(outputs, sales, stocks)
    loss.backward()
    optimizer.step()
    if (i + 1) % 100 == 0: # print in every 100 iterations
        #  after 2500 iterations there is a slight increase in loss function, which means gradient starts missing the optimum
        print("Epoch:", i, "Loss:", loss.item())

Epoch: 99 Loss: 0.6275261044502258
Epoch: 199 Loss: 0.5999700427055359
Epoch: 299 Loss: 0.5724143385887146
Epoch: 399 Loss: 0.5448583960533142
Epoch: 499 Loss: 0.5173026323318481
Epoch: 599 Loss: 0.4897467792034149
Epoch: 699 Loss: 0.4832867383956909
Epoch: 799 Loss: 0.4833013415336609
Epoch: 899 Loss: 0.48331600427627563
Epoch: 999 Loss: 0.483330637216568
Epoch: 1099 Loss: 0.48334527015686035
Epoch: 1199 Loss: 0.48335981369018555
Epoch: 1299 Loss: 0.4833745062351227
Epoch: 1399 Loss: 0.48338910937309265
Epoch: 1499 Loss: 0.48339301347732544
Epoch: 1599 Loss: 0.4833727478981018
Epoch: 1699 Loss: 0.4833526015281677
Epoch: 1799 Loss: 0.4833323359489441
Epoch: 1899 Loss: 0.48331207036972046
Epoch: 1999 Loss: 0.4832918345928192
Epoch: 2099 Loss: 0.48327162861824036
Epoch: 2199 Loss: 0.4832513928413391
Epoch: 2299 Loss: 0.48323118686676025
Epoch: 2399 Loss: 0.4832109808921814
Epoch: 2499 Loss: 0.48320674896240234
Epoch: 2599 Loss: 0.4832213521003723
Epoch: 2699 Loss: 0.4832359552383423
Epoc

# Further Findings:

```diff

@@ 1. First of all,  MeanModel is a linear model cannot capture a possible non-linearity between input and output. As such, this model doesn't actually consider any features of the input data and only learns a single, optimal mean value to minimize the custom loss function.

@@ 2. Another point to note is that the output of the model should be non-negative as it represents demand, which cannot be negative. However, the MeanModel doesn't guarantee non-negative outputs. Therefore, one can modify the model to ensure that it only produces non-negative outputs, for example by applying an activation function like ReLU.

@@ 3. Also, it may be beneficial to consider more complex models or additional features for the input if the performance of this simple MeanModel is not satisfactory.

@@ 4. This task only considers the problem of predicting a mean demand value, without any input features, and optimizing this prediction to minimize a custom loss function. In a real-world setting, however, it would be beneficial to consider additional features (such as past sales, time of year, promotions, etc.) and possibly a more complex model to better capture patterns in the data. Furthermore, performance should ideally be evaluated on a separate test set to ensure that the model can generalize well to new, unseen data.

```

```diff
@@ To ensure the model never generates negative values, we can modify the model to pass the output through an  activation function that constrains the output to positive values.

@@ One common choice for such an activation function is the ReLU (Rectified Linear Unit) function,  which sets all negative values to zero. However, in a forecasting task such as this, where the output  represents a count (number of items), we might prefer to have our model output be non-zero. In this case,  a better choice of activation function can be the Exponential function, which maps any real number to a positive value.


@@ Here is how we can modify the MeanModel to include an exponential activation function:
```

In [10]:
class NonNegativeMeanModel(nn.Module):
    def __init__(self):
        super(NonNegativeMeanModel, self).__init__()
        self.mean = nn.Parameter(torch.randn(1))
    
    def forward(self, n):
        return torch.exp(self.mean) * torch.ones(n)

In [11]:
# Train the model and optimize using gradient descent
model = NonNegativeMeanModel()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [12]:
for i in range(3000): # increase number of iterations from 100 to 3000
    optimizer.zero_grad()
    outputs = model(n)
    loss = forecast_loss(outputs, sales, stocks)
    loss.backward()
    optimizer.step()
    if (i + 1) % 100 == 0: # print in every 100 iterations
        print("Epoch:", i, "Loss:", loss.item())

Epoch: 99 Loss: 0.5052589178085327
Epoch: 199 Loss: 0.4833422303199768
Epoch: 299 Loss: 0.483344703912735
Epoch: 399 Loss: 0.48334720730781555
Epoch: 499 Loss: 0.48334962129592896
Epoch: 599 Loss: 0.4833521246910095
Epoch: 699 Loss: 0.4833545684814453
Epoch: 799 Loss: 0.4833570122718811
Epoch: 899 Loss: 0.4833594858646393
Epoch: 999 Loss: 0.4833618700504303
Epoch: 1099 Loss: 0.4833643436431885
Epoch: 1199 Loss: 0.48336678743362427
Epoch: 1299 Loss: 0.48336929082870483
Epoch: 1399 Loss: 0.483371764421463
Epoch: 1499 Loss: 0.4833741784095764
Epoch: 1599 Loss: 0.4833766520023346
Epoch: 1699 Loss: 0.483379065990448
Epoch: 1799 Loss: 0.48338156938552856
Epoch: 1899 Loss: 0.48338401317596436
Epoch: 1999 Loss: 0.48338645696640015
Epoch: 2099 Loss: 0.48338890075683594
Epoch: 2199 Loss: 0.4833914041519165
Epoch: 2299 Loss: 0.4833937883377075
Epoch: 2399 Loss: 0.4833962917327881
Epoch: 2499 Loss: 0.4833987355232239
Epoch: 2599 Loss: 0.48339658975601196
Epoch: 2699 Loss: 0.483393132686615
Epoch: 

```diff
@@ Now, the NonNegativeMeanModel should always output non-negative values regardless of the learned mean parameter, due to the properties of the exponential function. Please note that using exponential function can make the model harder to optimize and can lead to numerical instability if the values get too large. Hence, while training, we can monitor our loss values and consider decreasing the learning rate if we notice unusually large loss values or oscillations in the loss.
```